#### Endurance Strategy Quick Notebook

In [8]:
# ---------------------------------------------------
# INPUTS
# ---------------------------------------------------
# Dynamic race state
target_lap_time = 121.5      # <--- AVERAGE LAP TIME
laps_completed = 95          # <--- YOUR CURRENT LAP COUNT
leader_avg_lap_time = 120.8  # <--- LEADER AVG LAP TIME
leader_laps_completed = 98   # <--- LEADER LAP COUNT

# Race Information
race_distance_km = 1000
lap_length_km = 5.807

# Fuel Info
tank_capacity = 103.9
fuel_per_avg = 3.57
fuel_per_max = 3.67
fuel_per_min = 3.46

# Pit Stop Insights
total_pit_time = 62
service_time = 42
tyre_change_time = 21.0

In [9]:
# ---------------------------------------------------
# BASE CALCULATIONS
# ---------------------------------------------------
from math import ceil

# Calculate the total number of laps (UI = 173)
total_laps = ceil(race_distance_km / lap_length_km)
print(f"Total Laps: {total_laps}")

your_remaining = total_laps - laps_completed
leader_remaining = total_laps - leader_laps_completed
delta_laps = leader_laps_completed - laps_completed
print(f"Your Laps Completed: {laps_completed}")
print(f"Leader Laps Completed: {leader_laps_completed}")
print(f"Lap Delta (Leader - You): {delta_laps}")
print()
print(f"Your Laps Remaining: {your_remaining}")
print(f"Leader Laps Remaining: {leader_remaining}")
print()

# Pit Lane Loss (entry → stop → exit) minus the time spent refuelling/tyres
# i.e., the *fixed time* cost of visiting pit lane
pit_delta = total_pit_time - service_time
print(f"Pit Delta (fixed): {pit_delta:.1f}s")

# Refuelling Rate (litres per second)
refuelling_rate = tank_capacity / service_time
print(f"Refuelling Rate: {refuelling_rate:.3f} L/s")

# Litres that can be added during the tyre-change window
# During tyre change, fuel continues to flow — so this is "free fuel"
tyre_change_litres = ceil(tyre_change_time * refuelling_rate)

# Convert that amount of fuel into equivalent laps
tyre_change_stint = tyre_change_litres / fuel_per_avg

print(f"Tyre-Change 'Free Fuel': {tyre_change_litres:.2f} L "
      f"(≈ {tyre_change_stint:.2f} laps at avg burn)")

Total Laps: 173
Your Laps Completed: 95
Leader Laps Completed: 98
Lap Delta (Leader - You): 3

Your Laps Remaining: 78
Leader Laps Remaining: 75

Pit Delta (fixed): 20.0s
Refuelling Rate: 2.474 L/s
Tyre-Change 'Free Fuel': 52.00 L (≈ 14.57 laps at avg burn)


In [10]:
# ---------------------------------------------------
# STINT LENGTH CALCULATION
# ---------------------------------------------------
from math import floor

def laps_per_stint(burn):
    return floor(tank_capacity / burn)

laps_avg = laps_per_stint(fuel_per_avg)
laps_max = laps_per_stint(fuel_per_max)   # worst case
laps_min = laps_per_stint(fuel_per_min)   # best case

print("Fuel-Limited Stint Lengths:")
print(f"  AVG fuel burn: {laps_avg}")
print(f"  MAX fuel burn: {laps_max}")
print(f"  MIN fuel burn: {laps_min}")
print()

Fuel-Limited Stint Lengths:
  AVG fuel burn: 29
  MAX fuel burn: 28
  MIN fuel burn: 30



In [11]:
# ---------------------------------------------------
# REMAINING STINT PLANNING
# ---------------------------------------------------

def plan_remaining(laps_remaining, stint_size):
    stints = laps_remaining // stint_size
    last = laps_remaining % stint_size
    if last > 0:
        stints += 1
    return stints, last

# --- Your car ---
you_stints_avg, you_last_avg = plan_remaining(your_remaining, laps_avg)
you_stints_max, you_last_max = plan_remaining(your_remaining, laps_max)

# --- Leader ---
ldr_stints_avg, ldr_last_avg = plan_remaining(leader_remaining, laps_avg)
ldr_stints_max, ldr_last_max = plan_remaining(leader_remaining, laps_max)

print("Remaining Stints (You vs Leader)\n")

print("YOU (Avg fuel):")
print(f"  Stints Remaining: {you_stints_avg}")
print(f"  Last Stint Laps: {you_last_avg if you_last_avg else laps_avg}")

print("\nYOU (Max fuel):")
print(f"  Stints Remaining: {you_stints_max}")
print(f"  Last Stint Laps: {you_last_max if you_last_max else laps_max}")

print("\nLEADER (Avg fuel):")
print(f"  Stints Remaining: {ldr_stints_avg}")
print(f"  Last Stint Laps: {ldr_last_avg if ldr_last_avg else laps_avg}")

print("\nLEADER (Max fuel):")
print(f"  Stints Remaining: {ldr_stints_max}")
print(f"  Last Stint Laps: {ldr_last_max if ldr_last_max else laps_max}")
print()

Remaining Stints (You vs Leader)

YOU (Avg fuel):
  Stints Remaining: 3
  Last Stint Laps: 20

YOU (Max fuel):
  Stints Remaining: 3
  Last Stint Laps: 22

LEADER (Avg fuel):
  Stints Remaining: 3
  Last Stint Laps: 17

LEADER (Max fuel):
  Stints Remaining: 3
  Last Stint Laps: 19



In [12]:
# ---------------------------------------------------
# PIT STOP TIME QUICK ESTIMATES
# ---------------------------------------------------

def pit_time(tyres=True):
    return service_time + (tyre_change_time if tyres else 0)

print("Pit Times:")
print(f"  Fuel + Tyres: {pit_time(True)} s")
print(f"  Fuel Only:   {pit_time(False)} s")

Pit Times:
  Fuel + Tyres: 63.0 s
  Fuel Only:   42 s
